In [83]:
# ********** IMPORT LIBRARIES **************#


import numpy as np  # for handling multi-dimensional array operation
import pandas as pd  # for reading data from csv
#import statsmodels.api as sm  # for finding the p-value
from sklearn.preprocessing import MinMaxScaler  # for normalization
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle



In [86]:
# *********** Change all string feautres to numeric values ********

data=pd.read_csv('data/train.csv')
survival_map={1:1,0:-1}
gender_map={"male":1,"female":0}
data['Survived']=data["Survived"].map(survival_map)
data["Sex"]=data["Sex"].map(gender_map)

# data.drop(data.columns[[-1, 1]], axis=1, inplace=True)
data=data.drop(["Name",'SibSp',"Parch",'Ticket','Cabin',"Embarked"],axis=1)

